# Resnet18 without pretrain

In [1]:
from baseline_resnet18 import ResNet18
from Pretrain_resnet18 import clf_head
#Use the ResNet18 on Cifar-10
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torch
import torch.nn as nn

from torch.utils.tensorboard import SummaryWriter

from sklearn.metrics import accuracy_score,f1_score

In [2]:
#check gpu
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#set hyperparameter
EPOCH = 10
pre_epoch = 0
BATCH_SIZE = 50
LR = 0.0005

#prepare dataset and preprocessing
transform_train = transforms.Compose([
    transforms.Resize([128,128]),
    transforms.RandomCrop(128, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

transform_test = transforms.Compose([
    transforms.Resize([128,128]),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

trainset = torchvision.datasets.CIFAR10(root='../data_CIFAR10', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='../data_CIFAR10', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

#labels in CIFAR10
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


Files already downloaded and verified
Files already downloaded and verified


In [ ]:
#define ResNet18
baseline_model = ResNet18().to(device)
clf=clf_head(512,9,10).to(device)

#define loss funtion & optimizer
loss_f = nn.CrossEntropyLoss()
optimizer = optim.Adam([{'params': baseline_model.parameters()}, 
                        {'params': clf.parameters()}], lr=LR, weight_decay=5e-4)

writer = SummaryWriter('../logs/baseline_ResNet18')

In [3]:
# train
for epoch in range(pre_epoch, EPOCH):
    print('epoch:', epoch)
    print('Train start')

    baseline_model.train()
    clf.train()

    for i, data in enumerate(trainloader, 0):
        # prepare dataset
        length = len(trainloader)
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()

        # forward & backward
        outputs = baseline_model(inputs)
        outputs = clf(outputs)
        loss = loss_f(outputs, labels)
        loss.backward()
        optimizer.step()

        # print ac & loss in each batch
        with torch.no_grad():
            outputs_target = outputs.detach().cpu().numpy().argmax(axis=1)
            labels_check = labels.detach().cpu().numpy()
            acc_train = accuracy_score(labels_check, outputs_target)
            f1_train = f1_score(labels_check, outputs_target, average='macro')

            writer.add_scalar("Train/Loss",
                              loss.item(), (i + 1 + epoch * length))
            writer.add_scalar("Train/Acc",
                              acc_train, (i + 1 + epoch * length))
            writer.add_scalar("Train/F1_score(macro)",
                              f1_train, (i + 1 + epoch * length))

    # get the ac with testdataset in each epoch
    print('Train finish')

    print('Test start')
    with torch.no_grad():
        correct = 0
        total = 0
        for data in testloader:
            baseline_model.eval()
            clf.eval()
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = baseline_model(images)
            outputs = clf(outputs)
            predicted = outputs.detach().argmax(axis=1)

            total += labels.size(0)
            correct += (predicted == labels).sum()

        writer.add_scalar("Test/Acc",
                          correct/total, epoch)
    print('Test finish')


print('All train finish, Acc in test is ', (correct/total).item())

epoch: 0
Train start
Train finish
Test start
Test finish
epoch: 1
Train start
Train finish
Test start
Test finish
epoch: 2
Train start
Train finish
Test start
Test finish
epoch: 3
Train start
Train finish
Test start
Test finish
epoch: 4
Train start
Train finish
Test start
Test finish
epoch: 5
Train start
Train finish
Test start
Test finish
epoch: 6
Train start
Train finish
Test start
Test finish
epoch: 7
Train start
Train finish
Test start
Test finish
epoch: 8
Train start
Train finish
Test start
Test finish
epoch: 9
Train start
Train finish
Test start
Test finish
All train finish, Acc in test is  tensor(0.8241, device='cuda:0')


In [5]:
torch.save(baseline_model,'../model_path/resnet18.pt')
torch.save(clf,'../model_path/resnet18_clf_head.pt')
print('Without pretrain model saved')

Without pretrain model saved
